# Websites that I basically just copied/followed walkthrough from:
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [10]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from nltk.corpus import stopwords
import traitlets
from IPython.display import display
from ipywidgets import widgets
from tkinter import Tk, filedialog
import en_core_web_sm
nlp = en_core_web_sm.load()

In [2]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self, *args, **kwargs):
        """Initialize the SelectFilesButton class."""
        super(SelectFilesButton, self).__init__(*args, **kwargs)
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.
        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"
my_button = SelectFilesButton()
my_button

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [3]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
sentence_list = pd.read_csv(my_button.files[0])['Input.text'].tolist()

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(sentence_list))

In [6]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=10) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=10)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [7]:
trigram[data_words[125]]

['can_seem',
 'to',
 'get',
 'caught',
 'up',
 'on',
 'anything',
 'around',
 'the',
 'house']

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['bumper', 'bumper', 'traffic', 'drive', 'nut', 'ever', 'move']]


In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [14]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('bumper', 2),
  ('drive', 1),
  ('ever', 1),
  ('move', 1),
  ('nut', 1),
  ('traffic', 1)]]

In [15]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.426*"work" + 0.052*"go" + 0.036*"school" + 0.033*"try" + 0.020*"start" + '
  '0.019*"thing" + 0.017*"kid" + 0.016*"seem" + 0.016*"give" + 0.015*"be"'),
 (1,
  '0.209*"stress" + 0.080*"time" + 0.033*"find" + 0.032*"crazy" + '
  '0.025*"worry" + 0.024*"recent" + 0.020*"today" + 0.019*"take" + 0.019*"may" '
  '+ 0.018*"husband"'),
 (2,
  '0.116*"money" + 0.092*"boss" + 0.078*"stressful" + 0.062*"much" + '
  '0.038*"week" + 0.032*"deadline" + 0.031*"family" + 0.028*"stressed" + '
  '0.027*"life" + 0.026*"hour"'),
 (3,
  '0.160*"job" + 0.076*"lately" + 0.044*"worried" + 0.044*"make" + '
  '0.038*"situation" + 0.037*"day" + 0.028*"busy" + 0.024*"big" + 0.019*"help" '
  '+ 0.017*"enough"'),
 (4,
  '0.141*"get" + 0.087*"really" + 0.034*"want" + 0.033*"recently" + '
  '0.032*"right" + 0.031*"keep" + 0.025*"stuff" + 0.024*"new_job" + '
  '0.019*"break" + 0.017*"talk"'),
 (5,
  '0.040*"bad" + 0.032*"coworker" + 0.031*"buy" + 0.028*"year" + '
  '0.026*"figure" + 0.025*"enough_time" + 0.0

In [17]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.076014654506841

Coherence Score:  0.6182452580128681


In [18]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\simon\Anaconda3\Lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.396982 -0.005835       1        1  18.635874
1      0.059838  0.116097       2        1  15.312271
3      0.058597 -0.357049       3        1  14.015224
2      0.047539  0.130078       4        1  12.973998
4      0.048926  0.029976       5        1  11.256968
6      0.047180  0.025177       6        1   9.405962
8      0.047089  0.021420       7        1   7.075948
7      0.047516  0.026939       8        1   6.437212
5      0.040297  0.013196       9        1   4.886538, topic_info=     Category        Freq          Term       Total  loglift  logprob
8     Default  677.000000          work  677.000000  30.0000  30.0000
18    Default  272.000000        stress  272.000000  29.0000  29.0000
311   Default  192.000000           job  192.000000  28.0000  28.0000
26    Default  136.000000           get  136.000000  27.0000  27.0000
33    Default  129.000000         money  129.000000  26.0000  26.0000
776   Default  102.000000          boss  102.000000  25.0000  25.0000
63    Default   70.000000           lot   70.000000  24.0000  24.0000
97    Default  105.000000          time  105.000000  23.0000  23.0000
6     Default   84.000000        really   84.000000  22.0000  22.0000
21    Default   91.000000        lately   91.000000  21.0000  21.0000
188   Default   87.000000     stressful   87.000000  20.0000  20.0000
22    Default   70.000000          need   70.000000  19.0000  19.0000
819   Default   61.000000           pay   61.000000  18.0000  18.0000
47    Default   69.000000          much   69.000000  17.0000  17.0000
156   Default   58.000000          feel   58.000000  16.0000  16.0000
30    Default   84.000000            go   84.000000  15.0000  15.0000
789   Default   41.000000          bill   41.000000  14.0000  14.0000
70    Default   39.000000          know   39.000000  13.0000  13.0000
24    Default   53.000000       worried   53.000000  12.0000  12.0000
91    Default   53.000000          make   53.000000  11.0000  11.0000
23    Default   37.000000       problem   37.000000  10.0000  10.0000
347   Default   35.000000       project   35.000000   9.0000   9.0000
166   Default   41.000000           new   41.000000   8.0000   8.0000
174   Default   57.000000        school   57.000000   7.0000   7.0000
341   Default   38.000000         issue   38.000000   6.0000   6.0000
181   Default   45.000000     situation   45.000000   5.0000   5.0000
112   Default   45.000000           day   45.000000   4.0000   4.0000
41    Default   53.000000           try   53.000000   3.0000   3.0000
79    Default   42.000000          week   42.000000   2.0000   2.0000
239   Default   29.000000        people   29.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
16     Topic9   16.714806           bad   17.463476   2.9749  -3.2159
1148   Topic9   13.498721      coworker   14.247638   2.9647  -3.4296
662    Topic9   13.105228           buy   13.853990   2.9631  -3.4592
165    Topic9   11.793420          year   12.542149   2.9571  -3.5646
105    Topic9   10.869772        figure   11.618452   2.9521  -3.6462
224    Topic9   10.579543   enough_time   11.328806   2.9503  -3.6733
230    Topic9   10.255132         house   11.003771   2.9482  -3.7044
368    Topic9    9.170446          sick    9.919088   2.9402  -3.8162
930    Topic9    8.023034          save    8.771765   2.9295  -3.9499
296    Topic9    8.009490  relationship    8.758113   2.9293  -3.9516
499    Topic9    7.875589           dog    8.624246   2.9279  -3.9684
923    Topic9    7.356328         never    8.105097   2.9218  -4.0366
147    Topic9    7.238682          cook    7.987292   2.9203  -4.0527
11     Topic9    6.818148        dinner    7.566766   2.9145  -4.1126
406    Topic9    5.964634           see    6.713458   2.9004  -4.2463
533    Topic9    5.289069         avoid    6.038340   2.8862  -4.3665
490